In [1]:
import pydub
import derivative2_train_data  as derivative
import csv, datetime
import os, sys, glob, random
import numpy as np
import tensorflow as tf
import logging
from pydub.generators import WhiteNoise
import pydub
import typing
from typing import List, Dict, Union
import json
from tqdm.notebook import tqdm

2023-06-21 09:52:44.517139: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-21 09:52:45.492205: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [69]:
def generate_noisy_samples(input_file : str, num_samples : int, quiet_by_dB : int, output_dir : str, color : str = 'white', overlay : bool = True) -> typing.List[str]:
    '''
    Overlay sound samples with white noise for autoencoder training. 
    '''
    if not os.path.isdir(output_dir):
        os.makedirs(output_dir)
    prefix_output_file = os.path.join(output_dir, os.path.splitext(os.path.basename(input_file))[0])
    sample_mapping = []
    orig_sound = pydub.AudioSegment.from_file(input_file, format="flac")
    num = len(orig_sound.get_array_of_samples())
    frame_rate = orig_sound.frame_rate
    orig_sound = pydub.effects.normalize(orig_sound)
    audio_len  = len(orig_sound)  # Audio len in ms
    audio_half = int(0.5*audio_len)
    if overlay:
        for i in tqdm(range(num_samples)):
            marker1 = random.randint(0, audio_half)
            marker2 = marker1 + random.randint(audio_half, audio_len)
            marker2 = audio_len if marker2 > audio_len else marker2
            noise = generator.noise(num,color=color)
            noise = pydub.AudioSegment(noise.tobytes(), frame_rate = 48000, channels = 1, sample_width = 2)
            noise = noise - quiet_by_dB
            new_sound = orig_sound.overlay(noise)
            new_sound = new_sound.set_sample_width(2)
            new_sound_file = prefix_output_file + f"_{color}_" + str(i+1) + ".flac"
            sample_mapping.append(new_sound_file)
            new_sound.export(new_sound_file, format="flac")
    else:
        for i in tqdm(range(num_samples)): # Add no more than 50% of silence
            marker1 = random.randint(0, audio_half)
            marker2 = marker1 + random.randint(audio_half, audio_len)
            marker2 = audio_len if marker2 > audio_len else marker2
            noise = generator.noise(num,color=color)
            noise = pydub.AudioSegment(noise.tobytes(), frame_rate = 48000, channels = 1, sample_width = 2)
            noise = noise - quiet_by_dB
            new_sound = noise[:marker1] + orig_sound[marker1:marker2] + noise[marker2:audio_len]
            new_sound = new_sound.set_sample_width(2)
            new_sound_file = prefix_output_file + f"_{color}_snip_" + str(i+1) + ".flac"
            new_sound.export(new_sound_file, format="flac")
            sample_mapping.append(new_sound_file)
    return sample_mapping

In [78]:
def generate_noisy_dataset(input_files : List[str], num_samples : int, quiet_by_dB : int, output_dir : str, out_path : str, overlay : bool = True) -> Dict:
    '''
    generates between num_samples, num_samples*1.5 corrupted samples.
    also returns a JSON file I'll probably use later for writing generators.
    '''
    colors = ['white', 'pink', 'blue', 'brown', 'violet']
    mapping = {f : [] for f in input_files}
    for f in tqdm(input_files):
        num_samp = random.randint(num_samples//3, num_samples//2)
        for _ in range(num_samp):
            color = np.random.randint(0, len(colors))
            mapping[f].append(generate_noisy_samples(f, num_samp, quiet_by_dB, output_dir, colors[color], overlay))
    out_f = open(out_path, "w")
    json.dump(mapping, out_f)
    return mapping

In [79]:
generate_noisy_dataset(files, 5, 30, 'colored', 'lm.json')

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

KeyboardInterrupt: 

def generate_noisy_dataset(input_files: typing.List[str], )

##### SCRATCHHH


NameError: name 'generate_derivative_files' is not defined

In [74]:

INPUT_DIR_POS = '../data/true_samples'

In [73]:
import glob
files = glob.glob(f'{INPUT_DIR_POS}/*.flac')

In [9]:
files

['../data/OK/04-May-2022_03-53-21-539_GPIO_AUTO_TRIGGER.flac',
 '../data/OK/07-May-2022_15-12-10-878_GPIO_AUTO_TRIGGER.flac',
 '../data/OK/04-May-2022_03-47-20-557_GPIO_AUTO_TRIGGER.flac',
 '../data/OK/07-May-2022_15-22-41-849_GPIO_AUTO_TRIGGER.flac',
 '../data/OK/04-May-2022_03-57-12-228_GPIO_AUTO_TRIGGER.flac',
 '../data/OK/07-May-2022_15-16-00-809_GPIO_AUTO_TRIGGER.flac',
 '../data/OK/04-May-2022_03-29-44-585_GPIO_AUTO_TRIGGER.flac',
 '../data/OK/04-May-2022_03-42-27-593_GPIO_AUTO_TRIGGER.flac',
 '../data/OK/04-May-2022_03-38-47-546_GPIO_AUTO_TRIGGER.flac',
 '../data/OK/04-May-2022_03-33-34-003_GPIO_AUTO_TRIGGER.flac']

In [30]:
generate_noisy_samples(files[0], 5, 15, 'corrupted')

['corrupted/neg_4_gaussian_1.flac',
 'corrupted/neg_4_gaussian_2.flac',
 'corrupted/neg_4_gaussian_3.flac',
 'corrupted/neg_4_gaussian_4.flac',
 'corrupted/neg_4_gaussian_5.flac']

In [7]:
pydub.AudioSegment.from_file(pos_file)

In [23]:
generate_derivative_files(logger, pos_file, neg_files, 5, 15, 'try', [])

In [46]:
def invert(path1, path2):
    snd1 = pydub.AudioSegment.from_file(path1, format="flac")
    snd2 = pydub.AudioSegment.from_file(path2, format='flac')
    inverted = snd2.invert_phase()
    snd1 = snd1.overlay(inverted)
    path = ''.join(os.path.basename(path1).split('.')[0].split("_")[-2:]) + \
        ''.join(os.path.basename(path2).split('.')[0].split("_")[-2:]) + '.flac'
    snd1.export(path)
    return snd1

In [50]:
x = invert('try/04-May-2022_03-33-34-003_GPIO_AUTO_TRIGGER_gaussian_2.flac', 
           'try/04-May-2022_03-33-34-003_GPIO_AUTO_TRIGGER_gaussian_2.flac')

In [30]:
pos_file = '/home/juggernautjha/Desktop/to_rahul/maruti/data/OK/04-May-2022_03-33-34-003_GPIO_AUTO_TRIGGER.flac'

In [35]:
x.export("inverted.flac")

<_io.BufferedRandom name='inverted.flac'>

In [40]:
os.path.basename('try/04-May-2022_03-33-34-003_GPIO_AUTO_TRIGGER_gaussian_2.flac').split('.')[0].split("_")[-3:]

['TRIGGER', 'gaussian', '2']

##### PyAcoustics

In [12]:
from acoustics import generator

In [13]:
white = generator.noise_generator(color = 'white')

In [31]:
xx= pydub.AudioSegment.from_file(pos_file)

In [35]:
xx.frame_rate

48000

In [49]:
len(xx)

129984

In [32]:
len(xx.get_array_of_samples())

6239232

In [59]:
x = generator.noise(6239232//3, 'white')

In [60]:
n = pydub.AudioSegment(x.tobytes(), frame_rate = 48000, channels = 1, sample_width = 2)

In [61]:
len(n)

173312